In [ ]:
import imagehash
from PIL import Image
import os 
import numpy as np
import pandas as pd
from pathlib import Path
import glob
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import cv2

In [ ]:
# CONSTANTS AND PATHS
MERGE_DATA_PATH = './train/'
NEW_DATA_PATH   = "../cassava-leaf-disease-classification/train_images/"
OLD_DATA_PATH   = "../cassava-disease_old/"
ORDER           = True
NOT_SAVED_YET   = False

In [ ]:
# BASED ON https://github.com/JohannesBuchner/imagehash/blob/master/find_similar_images.py
if NOT_SAVED_YET:
    images = {}
    if ORDER:
        for img in tqdm(glob.glob(MERGE_DATA_PATH + '[0-9]*'), position=0, leave=True):
            try:
                hash = imagehash.average_hash(Image.open(img))
            except Exception as e:
                print('Problem:', e, 'with', img)
                continue

            images[hash] = images.get(hash, []) + [img]

        print('FINISHED FOR NEW DATA SET')

        for img in tqdm(glob.glob(MERGE_DATA_PATH + 'train*'), position=0, leave=True):
            try:
                hash = imagehash.average_hash(Image.open(img))
            except Exception as e:
                print('Problem:', e, 'with', img)
                continue

            images[hash] = images.get(hash, []) + [img]

        print('WENT THROUGH OLD DATA SET')
    else:
        for img in tqdm(glob.glob(MERGE_DATA_PATH + '*.jpg'), position=0, leave=True):
            try:
                hash = imagehash.average_hash(Image.open(img))
            except Exception as e:
                print('Problem:', e, 'with', img)
                continue
            if hash in images:
                print(img, '  already exists as', ' '.join(images[hash]))
            images[hash] = images.get(hash, []) + [img]

    dupl_list = [value for value in images.values() if len(value) > 1]

    with open('duplicate_list.txt', 'w') as f:
        for item in dupl_list:
            f.write("%s\n" % item)

    with open('dupl_pickle.pkl', 'wb') as f:
           pickle.dump(dupl_list, f)
            
else:
    with open('dupl_pickle.pkl', 'rb') as f:
           dupl_list = pickle.load(f)

In [ ]:
for i, element in tqdm(enumerate(dupl_list), position=0, leave=True):
    plt.figure(figsize=(20,50))
    fig, axs = plt.subplots(1, len(element))
    fig.suptitle(f'Line {i}')
    for i, img_name in enumerate(element): 
        img = cv2.cvtColor(cv2.imread(img_name), cv2.COLOR_BGR2RGB)
        axs[i].imshow(img)

In [ ]:
lines_dropped = [elem for i, elem in enumerate(dupl_list) if i not in [87, 112, 119, 120, 121, 126, 140, 230, 232, 264]]
strange_dups = [elem for i, elem in enumerate(dupl_list) if i in [87, 112, 119, 120, 121, 126, 140, 230, 232, 264]]

In [ ]:
with open('dupl_pickle_lines_dropped.pkl', 'wb') as f:
           pickle.dump(lines_dropped, f)

In [ ]:
print(lines_dropped)